# Azure AI Agent service - Function calling

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [ ]:
#%pip install azure-ai-projects
#%pip install yfinance

In [ ]:
import glob
import json
import ipyplot
import matplotlib.pyplot as plt
import os
import requests
import sys
import time
import yfinance as yf

#from azure.ai.projects import AIProjectClient
from azure.ai.agents import AgentsClient 
from azure.identity import DefaultAzureCredential
#from azure.ai.projects.models import FunctionTool, ToolSet, CodeInterpreterTool
from azure.ai.agents.models import (
    FunctionTool,
    ListSortOrder,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ToolOutput,
)

from dotenv import load_dotenv

load_dotenv()
from user_functions import user_functions, azuremaps_weather

## 1. Agent definition - Weather forecasts

## Connect to AI Foundry Project

In [ ]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

### Testing the Azure Maps Weather function

We will use the Azure Maps service to provide weather information.

In [3]:
weather = azuremaps_weather("Toronto")
json.loads(weather)

{'weather_data': {'results': [{'dateTime': '2025-06-18T11:07:00-04:00',
    'phrase': 'Partly sunny',
    'iconCode': 3,
    'hasPrecipitation': False,
    'isDayTime': True,
    'temperature': {'value': 22.2, 'unit': 'C', 'unitType': 17},
    'realFeelTemperature': {'value': 26.1, 'unit': 'C', 'unitType': 17},
    'realFeelTemperatureShade': {'value': 21.3, 'unit': 'C', 'unitType': 17},
    'relativeHumidity': 88,
    'dewPoint': {'value': 20.0, 'unit': 'C', 'unitType': 17},
    'wind': {'direction': {'degrees': 203.0, 'localizedDescription': 'SSW'},
     'speed': {'value': 9.3, 'unit': 'km/h', 'unitType': 7}},
    'windGust': {'speed': {'value': 9.3, 'unit': 'km/h', 'unitType': 7}},
    'uvIndex': 6,
    'uvIndexPhrase': 'High',
    'visibility': {'value': 11.3, 'unit': 'km', 'unitType': 6},
    'obstructionsToVisibility': '',
    'cloudCover': 35,
    'ceiling': {'value': 244.0, 'unit': 'm', 'unitType': 5},
    'pressure': {'value': 1010.3, 'unit': 'mb', 'unitType': 14},
    'pressu

### Agent

In [4]:
model = "gpt-4o-mini"
name = "gpt-4o-mini-weather-agent"
instructions = "You are a weather bot. Use the provided functions to help answer questions."

functions = FunctionTool(user_functions)

# Create an agent and run user's request with function calls
agent = project_client.create_agent(
    model=os.getenv("MODEL_DEPLOYMENT_NAME"),
    name=name,
    instructions=instructions,
    tools=functions.definitions,
)
print(f"Created agent, ID: {agent.id}")



Created agent, ID: asst_ozrvk8zvOTwKXn1YHWjA8Wt2


### Tests

In [5]:
prompt = "Hello, generate a full report for the weather in Toronto today"

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_6IeKt2CCiSBoH3BcRYZmq5Ht
Created message ID = msg_L8BVYVvF7PpyZiTafCacZPxn


In [6]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_P3kK36kertnvZV6vUKKuhw99


In [7]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_SL3UAcEyGTl4ZU7iL3VxShL3', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Toronto"}'}}
Tool outputs: [{'tool_call_id': 'call_SL3UAcEyGTl4ZU7iL3VxShL3', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-18T11:07:00-04:00", "phrase": "Partly sunny", "iconCode": 3, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 22.2, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 26.1, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 21.3, "unit": "C", "unitType": 17}, "relativeHumidity": 88, "dewPoint": {"value": 20.0, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 203.0, "localizedDescription": "SSW"}, "speed": {"value": 9.3, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 9.3, "unit": "km/h", "unitType": 7}}, "uvIndex": 6, "uvIndexPhrase": "High", "visibility": {"value": 11.3, "unit": "km", "unitType": 6}, "obstructionsToVisib

In [8]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-18 15:16:15+0000
End     : 2025-06-18 15:16:19+0000
Elapsed : 0:00:04  (4.00 seconds)


{'prompt_tokens': 1427, 'completion_tokens': 327, 'total_tokens': 1754, 'prompt_token_details': {'cached_tokens': 0}}

In [9]:
# Fetch and log all messages
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Hello, generate a full report for the weather in Toronto today
MessageRole.AGENT: Here is the full weather report for Toronto today:

Current Conditions:

- Description: Partly sunny
- Temperature: 22.2°C (feels like 26.1°C)
- Temperature in the shade: 21.3°C
- Minimum temperature (last 24h): 17.8°C
- Maximum temperature (last 24h): 23.5°C
- Relative Humidity: 88%
- Dew Point: 20.0°C
- Wind: 9.3 km/h from the south-southwest (SSW)
- Wind Gusts: 9.3 km/h
- UV Index: 6 (High)
- Cloud Cover: 35%
- Visibility: 11.3 km
- Atmospheric Pressure: 1010.3 mb (steady)

Precipitation:

- Past hour: 0.0 mm
- Past 3–24 hours: 1.0 mm (total in each interval)

Additional Information:

- Wet Bulb Temperature: 20.8°C
- No significant obstructions to visibility
- No wind chill; apparent temperature: 22.8°C

Summary: Today in Toronto is partly sunny with mild temperatures, moderate humidity, and light winds. There has been a small amount of rain over the past 24 hours (1.0 mm), but curren

### Other examples

In [10]:
prompt = "Hello, generate a full report for the weather in Mexico City today. Print the results in English, Spanish and French."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_8BHEvQK7rO3Yi1o0U5yvLT3t
Created message ID = msg_4KyHOSMqRJIHkOLnCEMstivg


In [11]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_Cm9mgFRdVRHJElPhkcYst0xZ


In [12]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_Tt9YGjG2KP7y1ubbi0MjMlHH', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Mexico City"}'}}
Tool outputs: [{'tool_call_id': 'call_Tt9YGjG2KP7y1ubbi0MjMlHH', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-18T08:55:00-06:00", "phrase": "Cloudy", "iconCode": 7, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 16.1, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 16.7, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 16.7, "unit": "C", "unitType": 17}, "relativeHumidity": 77, "dewPoint": {"value": 12.2, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 0.0, "localizedDescription": "N"}, "speed": {"value": 7.4, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 7.4, "unit": "km/h", "unitType": 7}}, "uvIndex": 1, "uvIndexPhrase": "Low", "visibility": {"value": 8.0, "unit": "km", "unitType": 6}, "obstructionsToVisibility": 

In [13]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-18 15:17:21+0000
End     : 2025-06-18 15:17:25+0000
Elapsed : 0:00:04  (4.00 seconds)


{'prompt_tokens': 1450, 'completion_tokens': 470, 'total_tokens': 1920, 'prompt_token_details': {'cached_tokens': 0}}

In [14]:
# Fetch and log all messages
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Hello, generate a full report for the weather in Mexico City today. Print the results in English, Spanish and French.
MessageRole.AGENT: Weather Report for Mexico City Today (English):

- Condition: Cloudy
- Temperature: 16.1°C (Feels like 16.7°C)
- Humidity: 77%
- Dew Point: 12.2°C
- Wind: North at 7.4 km/h
- Visibility: 8.0 km
- UV Index: 1 (Low)
- Pressure: 1025.1 mb (Rising)
- Cloud Cover: 100%
- Precipitation (last 24 hrs): 6.7 mm
- Minimum/Maximum Temperature (last 24 hrs): 13.9°C / 20.0°C

Reporte del clima para la Ciudad de México hoy (Español):

- Condición: Nublado
- Temperatura: 16.1°C (Sensación térmica de 16.7°C)
- Humedad: 77%
- Punto de rocío: 12.2°C
- Viento: Norte a 7.4 km/h
- Visibilidad: 8.0 km
- Índice UV: 1 (Bajo)
- Presión: 1025.1 mb (En aumento)
- Nubosidad: 100%
- Precipitación (últimas 24 hrs): 6.7 mm
- Temperatura mínima/máxima (últimas 24 hrs): 13.9°C / 20.0°C

Bulletin météo pour Mexico aujourd'hui (Français) :

- Condition : Nuageux
- Temp

In [15]:
prompt = "Hello, print the UV informations for Oakville, Ontario today. Print the date as well of the weather data."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_uUN963pb8aGQWvqywTtIX6hJ
Created message ID = msg_vnrOTnGVF5uxXzo20Q28Hf67


In [16]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_fgkemeBbJxFNGYskNhxJOfLU


In [17]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_PQRCuG3ZaOt0RM7C2yqsPhxo', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Oakville, Ontario"}'}}
Tool outputs: [{'tool_call_id': 'call_PQRCuG3ZaOt0RM7C2yqsPhxo', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-18T11:07:00-04:00", "phrase": "Mostly cloudy", "iconCode": 6, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 25.5, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 31.8, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 29.3, "unit": "C", "unitType": 17}, "relativeHumidity": 85, "dewPoint": {"value": 22.9, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 225.0, "localizedDescription": "SW"}, "speed": {"value": 2.8, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 5.3, "unit": "km/h", "unitType": 7}}, "uvIndex": 3, "uvIndexPhrase": "Moderate", "visibility": {"value": 12.9, "unit": "km", "unitType": 6}, "obstr

In [18]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-18 15:18:11+0000
End     : 2025-06-18 15:18:12+0000
Elapsed : 0:00:01  (1.00 seconds)


{'prompt_tokens': 1452, 'completion_tokens': 86, 'total_tokens': 1538, 'prompt_token_details': {'cached_tokens': 0}}

In [19]:
# Fetch and log all messages
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Hello, print the UV informations for Oakville, Ontario today. Print the date as well of the weather data.
MessageRole.AGENT: Date: 2025-06-18

UV Index information for Oakville, Ontario today:
- UV Index: 3
- UV Index Level: Moderate

A UV Index of 3 means moderate risk from unprotected sun exposure, so it's a good idea to take basic sun safety precautions if you'll be outdoors.


In [20]:
prompt = "Generate a weather report for Rome. This report is for a blog publication. Add emojis and adopt a fun style."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_Q5ZpKwhuwgNPEUBYxBZaw6tP
Created message ID = msg_VdG0bNUDMFiTAD1XVTdBX1q6


In [21]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_THBZkRzNuM59o9UPi2JVcMxF


In [22]:
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

Executing tool call: {'id': 'call_GW6be3SW7vt3XzXEZrekR1T1', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Rome"}'}}
Tool outputs: [{'tool_call_id': 'call_GW6be3SW7vt3XzXEZrekR1T1', 'output': '{"weather_data": {"results": [{"dateTime": "2025-06-18T17:07:00+02:00", "phrase": "Mostly sunny", "iconCode": 2, "hasPrecipitation": false, "isDayTime": true, "temperature": {"value": 29.1, "unit": "C", "unitType": 17}, "realFeelTemperature": {"value": 30.9, "unit": "C", "unitType": 17}, "realFeelTemperatureShade": {"value": 29.6, "unit": "C", "unitType": 17}, "relativeHumidity": 63, "dewPoint": {"value": 21.5, "unit": "C", "unitType": 17}, "wind": {"direction": {"degrees": 180.0, "localizedDescription": "S"}, "speed": {"value": 17.5, "unit": "km/h", "unitType": 7}}, "windGust": {"speed": {"value": 30.7, "unit": "km/h", "unitType": 7}}, "uvIndex": 2, "uvIndexPhrase": "Low", "visibility": {"value": 25.7, "unit": "km", "unitType": 6}, "obstructionsToVisibilit

In [23]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-18 15:18:57+0000
End     : 2025-06-18 15:19:03+0000
Elapsed : 0:00:06  (6.00 seconds)


{'prompt_tokens': 1448, 'completion_tokens': 254, 'total_tokens': 1702, 'prompt_token_details': {'cached_tokens': 0}}

In [24]:
# Fetch and log all messages
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

MessageRole.USER: Generate a weather report for Rome. This report is for a blog publication. Add emojis and adopt a fun style.
MessageRole.AGENT: 🇮🇹🌞 Buongiorno from Rome, the Eternal City! 

Today’s weather is absolutely bellissimo: mostly sunny skies are lighting up the ancient streets, with just a few clouds doing their best to photobomb your Colosseum selfies. Temperatures are feeling fabulous at 29°C (that’s 84°F for our international amici), but trust us—it feels a spicy 31°C ☀️🔥. It’s the perfect weather for a scoop (or three) of gelato!

Humidity is hanging around at 63%—so you might feel a bit like a fresh mozzarella, but hey, that's all part of the Roman holiday experience! A gentle southern breeze at 17 km/h is swirling through, great for keeping your cappuccino cool at the café.

There’s absolutely no rain in sight, UV index is low, and visibility is crystal clear. Rome is ready for your wandering! 👒🕶️ Don’t forget to toss a coin in the Trevi Fountain—it just might stay sun

### Post processing

In [ ]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

In [ ]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")

In [ ]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")